<a href="https://colab.research.google.com/github/leonardoub/cmepda/blob/master/BRATS_nested_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy

#Load data

In [2]:
#load data from Drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
dataset_path = '/gdrive/My Drive/BRATS/data_without_NAN_with_histologies.csv'

In [0]:
df_data = pd.read_csv(dataset_path)

In [5]:
df_data

Unnamed: 0            ID        Date  ...  TGM_Cog_Z_1    TGM_T_1  Histology
0             0  TCGA-02-0006  1996.08.23  ...     0.052741    2.00000        GBM
1             1  TCGA-02-0009  1997.06.14  ...     0.094336   91.47360        GBM
2             2  TCGA-02-0011  1998.02.01  ...     0.096035  272.42900        GBM
3             3  TCGA-02-0027  1999.03.28  ...     0.096470  128.46800        GBM
4             4  TCGA-02-0033  1997.05.26  ...     0.096894  240.77800        GBM
..          ...           ...         ...  ...          ...        ...        ...
141         141  TCGA-HT-7694  1995.04.04  ...     0.090456  719.23800        LGG
142         142  TCGA-HT-8018  1997.04.11  ...     0.054307    2.00000        LGG
143         143  TCGA-HT-8111  1998.03.30  ...     0.126712    7.06744        LGG
144         144  TCGA-HT-8114  1998.10.30  ...     0.107433   15.52240        LGG
145         145  TCGA-HT-8563  1998.12.09  ...     0.069101    7.62280        LGG

[146 rows x 708 columns]

In [7]:
df_data.columns

Index(['Unnamed: 0', 'ID', 'Date', 'VOLUME_ET', 'VOLUME_NET', 'VOLUME_ED',
       'VOLUME_TC', 'VOLUME_WT', 'VOLUME_BRAIN', 'VOLUME_ET_OVER_NET',
       ...
       'TEXTURE_NGTDM_NET_FLAIR_Busyness',
       'TEXTURE_NGTDM_NET_FLAIR_Complexity',
       'TEXTURE_NGTDM_NET_FLAIR_Strength', 'TGM_p1', 'TGM_dw', 'TGM_Cog_X_1',
       'TGM_Cog_Y_1', 'TGM_Cog_Z_1', 'TGM_T_1', 'Histology'],
      dtype='object', length=708)

In [0]:
data = df_data.drop(['Histology', 'Unnamed: 0', 'ID', 'Date'], axis=1)

In [0]:
labels = df_data.Histology

#Random forest

In [0]:
from sklearn import datasets
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV,KFold,cross_val_predict,cross_val_score,StratifiedKFold

from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import classification_report,accuracy_score

In [0]:
#Vettorizzare i label
encoder = LabelEncoder()
labels_encoded = encoder.fit_transform(labels)

In [0]:
#Scalers
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
scalers_to_test = [StandardScaler(), RobustScaler(), MinMaxScaler(), None]

In [0]:
# Designate distributions to sample hyperparameters from 
n_tree = [10, 50, 70, 100, 150]
n_features_to_test = [0.85, 0.9, 0.95]
depth = [10, 40, 50, 100, None]

In [0]:
# Choose cross-validation techniques for the inner and outer loops,
# independently of the dataset.
# E.g "GroupKFold", "LeaveOneOut", "LeaveOneGroupOut", etc.
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)
outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=1)

In [0]:
#RandomForestClassifier
steps = [('scaler', StandardScaler()), ('red_dim', PCA()), ('clf', RandomForestClassifier(random_state=503))]
pipeline = Pipeline(steps)

parameteres = [{'scaler':[StandardScaler()], 'red_dim':[PCA(random_state=42)], 'red_dim__n_components':list(n_features_to_test),
                'red_dim__whiten':[False, True], 
                'clf__n_estimators':list(n_tree), 'clf__criterion':['gini', 'entropy'], 
                'clf__max_depth':depth, 'clf__min_samples_split':[2, 5], 
                'clf__min_samples_leaf':[2, 4], 'clf__class_weight':[None, 'balanced']}]

In [0]:
grid = GridSearchCV(pipeline, param_grid=parameteres, cv=inner_cv, n_jobs=-1, scoring='roc_auc', verbose=1)

In [40]:
# Nested CV with parameter optimization
nested_score = cross_val_score(grid, X=data, y=labels_encoded, cv=outer_cv)


Fitting 3 folds for each of 2400 candidates, totalling 7200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 372 tasks      | elapsed:   38.9s
[Parallel(n_jobs=-1)]: Done 872 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1572 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 2472 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3572 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 4872 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 6372 tasks      | elapsed: 11.1min
[Parallel(n_jobs=-1)]: Done 7200 out of 7200 | elapsed: 12.6min finished


Fitting 3 folds for each of 2400 candidates, totalling 7200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 124 tasks      | elapsed:   10.9s
[Parallel(n_jobs=-1)]: Done 424 tasks      | elapsed:   41.9s
[Parallel(n_jobs=-1)]: Done 924 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done 1624 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 2524 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3624 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 4924 tasks      | elapsed:  8.5min
[Parallel(n_jobs=-1)]: Done 6424 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 7200 out of 7200 | elapsed: 12.6min finished


Fitting 3 folds for each of 2400 candidates, totalling 7200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 110 tasks      | elapsed:    9.9s
[Parallel(n_jobs=-1)]: Done 410 tasks      | elapsed:   40.3s
[Parallel(n_jobs=-1)]: Done 910 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 1610 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 2510 tasks      | elapsed:  4.3min
[Parallel(n_jobs=-1)]: Done 3610 tasks      | elapsed:  6.3min
[Parallel(n_jobs=-1)]: Done 4908 tasks      | elapsed:  8.6min
[Parallel(n_jobs=-1)]: Done 6408 tasks      | elapsed: 11.2min
[Parallel(n_jobs=-1)]: Done 7200 out of 7200 | elapsed: 12.6min finished


In questo modo non riesco a sapere quali sono i best HP

In [41]:
nested_score

array([0.87647059, 0.90784314, 0.88865546])

In [23]:


df = pd.DataFrame()



for i in range(1, 6):

    #Train test split
    X_train, X_test, y_train, y_test = train_test_split(public_data, public_labels, test_size=0.3, 
    stratify=public_labels, random_state=i*500)

    #Vettorizzare i label
    train_labels_encoded = encoder.fit_transform(y_train)
    test_labels_encoded = encoder.transform(y_test)

    #RandomForestClassifier
    steps = [('scaler', StandardScaler()), ('red_dim', PCA()), ('clf', RandomForestClassifier(random_state=i*503))]

    pipeline = Pipeline(steps)

    parameteres = [{'scaler':scalers_to_test, 'red_dim':[PCA(random_state=42)], 'red_dim__n_components':list(n_features_to_test),
                    'red_dim__whiten':[False, True], 
                    'clf__n_estimators':list(n_tree), 'clf__criterion':['gini', 'entropy'], 
                    'clf__max_depth':depth, 'clf__min_samples_split':[2, 5, 10], 
                    'clf__min_samples_leaf':[1, 2, 4], 'clf__class_weight':[None, 'balanced']}]

    grid = GridSearchCV(pipeline, param_grid=parameteres, cv=3, n_jobs=-1, verbose=1)



     grid.fit(X_train, y_train)

    score_train = grid.score(X_train, y_train)
    score_test = grid.score(X_test, y_test)
    best_p = grid.best_params_

    bp = pd.DataFrame(best_p, index=[i])
    bp['accuracy_train'] = score_train
    bp['accuracy_test'] = score_test
    bp['random_state'] = i*500
    bp['random_state_pca'] = i*42
    bp['random_state_clf'] = i*503

    df = df.append(bp, ignore_index=True)


NameError: ignored